In [4]:
import pandas as pd
import numpy as np
import os

# Get the current working directory
directory = os.getcwd()

# Initialize an empty list to store DataFrames for each CSV file
dataframes = []

csv_files_order = [
    "abcd_p_demo.csv",
    "mh_p_fhx.csv",
    "mh_p_ksads_gad.csv",
    "mh_p_ksads_pd.csv",
    "mh_p_ksads_sad.csv",
    "mh_y_ksads_gad.csv",
    "mh_y_ksads_sad.csv",
    "mh_y_pps.csv",
    "mri_y_smr_area_dsk.csv",
    "mri_y_smr_area_dst.csv",
    "mri_y_smr_sulc_dsk.csv",
    "mri_y_smr_sulc_dst.csv",
    "mri_y_smr_thk_dsk.csv",
    "mri_y_smr_thk_dst.csv",
    "mri_y_smr_vol_aseg.csv",
    "mri_y_smr_vol_dsk.csv",
    "mri_y_smr_vol_dst.csv",
    "mri_y_dti_fa_is_at.csv",
    "mri_y_dti_ld_is_at.csv",
    "mri_y_dti_md_is_at.csv",
    "mri_y_dti_td_is_at.csv",
    "RVI_groups.csv"
]

def count_mri_missing_data(df, filename) -> str:
    # Count the number of rows with missing data
    missing_rows_count = df.isnull().any(axis=1).sum()
    total = len(df)
    return(f"Missing: {missing_rows_count}. Number of rows without missing data in {filename}: {total - missing_rows_count}")


def drop_rows_with_missing_data(df, filename):
    # Drop rows with any missing value in any column
    og_length = len(df)

    df.replace('NA', np.nan, inplace=True)
    df = df.dropna(axis=0)
    
    # Print info about the deleted rows
    deleted_rows_count = og_length - len(df)
    print(f"Deleted {deleted_rows_count} rows with missing data from {filename}.")
    
    return df


def delete_specific_columns(df, columns):
    # Delete specific columns from the DataFrame
    df = df.drop(columns=columns, axis=1)
    return df

# Define criteria for in distribution vs out of distribution
def typical(row):
    if row['famhx_ss_fath_prob_vs_p'] == 0 and row['famhx_ss_moth_prob_vs_p'] == 0 and row['famhx_ss_fath_prob_nrv_p'] == 0 and row['famhx_ss_moth_prob_nrv_p'] == 0 \
        and row['ksads_gad_raw_271_p'] < 1 and row['ksads_gad_raw_273_p'] == 0 \
        and row['ksads_pd_raw_176_p'] < 1 and row['ksads_pd_raw_178_p'] == 0 \
        and row['ksads_sad_raw_209_p'] < 1 and row['ksads_sad_raw_211_p'] == 0 \
        and row['ksads_gad_raw_271_t'] < 1 and row['ksads_gad_raw_273_t'] == 0 \
        and row['ksads_sad_raw_209_t'] < 1 and row['ksads_sad_raw_211_t'] == 0 \
        and row['group_last_final'] == 5 \
        and row['pps_y_ss_number'] <= 1 and (row['pps_y_ss_bother_sum'] <= 1 or (row['pps_y_ss_number'] <= 1 and pd.isna(row['pps_y_ss_bother_sum']))):

        return 0 #squeeky clean
    
    elif row['pps_y_ss_number'] > 2 and row['pps_y_ss_bother_sum'] > 2 and row['group_last_final'] == 1:
        
        return 2 #outlier
    
    else:
        return 1 #"typical"


# Iterate through each CSV file in the directory
for filename in csv_files_order:
    if filename in os.listdir(directory):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(directory, filename))
        
        # Filter rows with "baseline" for the event name column
        if 'eventname' in df.columns:
            df = df[df['eventname'].str.startswith("baseline")]

        # Select columns you want to keep and check amounts of usable data
        if 'abcd_p_demo.csv' in filename:
            df = df[['src_subject_id', 'eventname', 'demo_brthdat_v2']] 
            print(count_mri_missing_data(df, filename)) 
            df = drop_rows_with_missing_data(df, filename)
        
        if 'mh_p_cbcl.csv' in filename:
            exit

        if 'RVI_groups.csv' in filename:
            df = df.rename(columns={'ID': 'src_subject_id'})
            df = df[['src_subject_id', 'group_last_final']]
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)
        
        if 'mh_p_fhx.csv' in filename:
            df = df[['src_subject_id','famhx_ss_fath_prob_vs_p', 'famhx_ss_moth_prob_vs_p','famhx_ss_fath_prob_nrv_p','famhx_ss_moth_prob_nrv_p']]  
            df = drop_rows_with_missing_data(df, filename)


        if 'mh_p_ksads_gad.csv' in filename:
            df = df[['src_subject_id', 'ksads_gad_raw_271_p', 'ksads_gad_raw_273_p']]
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mh_p_ksads_pd.csv' in filename:
            df = df[['src_subject_id', 'ksads_pd_raw_176_p', 'ksads_pd_raw_178_p']]
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mh_p_ksads_sad.csv' in filename:
            df = df[['src_subject_id', 'ksads_sad_raw_209_p', 'ksads_sad_raw_211_p']]
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mh_y_ksads_gad.csv' in filename:
            df = df[['src_subject_id', 'ksads_gad_raw_271_t', 'ksads_gad_raw_273_t']]
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)
        
        if 'mh_y_ksads_sad.csv' in filename:
            df = df[['src_subject_id', 'ksads_sad_raw_209_t', 'ksads_sad_raw_211_t']]
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)
            sum_per_entry = df.groupby('ksads_sad_raw_209_t').size()
            print(sum_per_entry)
        
        if 'mh_y_pps.csv' in filename:
            df = df[['src_subject_id', 'pps_y_ss_number', 'pps_y_ss_bother_sum']]
            mask = (df['pps_y_ss_number'] == 0) & (df['pps_y_ss_bother_sum'].isna())
            df.loc[mask, 'pps_y_ss_bother_sum'] = 0
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)
            sum_per_entry = df.groupby('pps_y_ss_bother_sum').size()
            print(sum_per_entry)
            sum_per_entry = df.groupby('pps_y_ss_number').size()
            print(sum_per_entry)
        
        if 'mri_y_dti_fa_is_at.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)
        
        if 'mri_y_dti_ld_is_at.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_dti_md_is_at.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_dti_td_is_at.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_area_dsk.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_area_dst.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_sulc_dsk.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_sulc_dst.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_thk_dsk.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_thk_dst.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_vol_aseg.csv' in filename:
            df = delete_specific_columns(df, columns=['smri_vol_scs_lesionlh', 'smri_vol_scs_lesionrh'])
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_vol_dsk.csv' in filename:
            print(count_mri_missing_data(df, filename))
            df = drop_rows_with_missing_data(df, filename)

        if 'mri_y_smr_vol_dst.csv' in filename:
           print(count_mri_missing_data(df, filename))
           df = drop_rows_with_missing_data(df, filename)

        
        # Append the processed DataFrame to the list
       
        dataframes.append(df)
        
        

#Concatenate all DataFrames into a single DataFrame
merged_data = pd.concat(dataframes, ignore_index=True)

# Group by subject ID and aggregate the rows using a function (e.g., first)
merged_data = merged_data.groupby('src_subject_id').first().reset_index()



merged_data = drop_rows_with_missing_data(merged_data, 'merged_data.csv')

merged_data = merged_data.drop(columns=['src_subject_id', 'eventname'])
# Calculate mean and standard deviation of 'pps_y_ss_number' column

# numpy estima lambda value to find mean and standard deviation. 
# mean_pps = merged_data['pps_y_ss_number'].mean()
# std_pps = merged_data['pps_y_ss_number'].std()

# mean_pps_bother = merged_data['pps_y_ss_bother_sum'].mean()
# std_pps_bother = merged_data['pps_y_ss_bother_sum'].std()

# print(mean_pps, std_pps, mean_pps_bother, std_pps_bother)

# Define threshold for distribution
# in_threshold = mean_pps - 0.5 * std_pps
# in_threshold_bother = mean_pps_bother - 0.5 * std_pps_bother


# Apply the criteria to each row and create a new column
merged_data['Distribution Status'] = merged_data.apply(typical, axis=1)


# Calculate the sum of all values in the "Distribution Status" column

squeeky = (merged_data['Distribution Status'] == 0).sum()
ood_total = (merged_data['Distribution Status'] == 1).sum()
outlier = (merged_data['Distribution Status'] == 2).sum()
# Add a new column with the sum as its value for all rows
# merged_data['Total Distribution Status'] = ood_total

print("Squeeky clean, ood, outliers:", squeeky, ood_total, outlier)


# # Write the merged DataFrame to a new CSV file
merged_data.to_csv(os.path.join(directory, 'merged_data.csv'), index=False)


# # Now you have all your data merged into one DataFrame


/var/folders/b2/_pxlc3ws1rnfp02p32rld8tr0000gn/T/ipykernel_810/2233842345.py:87: DtypeWarning: Columns (275) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename))


Missing: 11. Number of rows without missing data in abcd_p_demo.csv: 11857
Deleted 11 rows with missing data from abcd_p_demo.csv.
Deleted 864 rows with missing data from mh_p_fhx.csv.
Missing: 320. Number of rows without missing data in mh_p_ksads_gad.csv: 11317
Deleted 320 rows with missing data from mh_p_ksads_gad.csv.
Missing: 56. Number of rows without missing data in mh_p_ksads_pd.csv: 11487
Deleted 56 rows with missing data from mh_p_ksads_pd.csv.
Missing: 203. Number of rows without missing data in mh_p_ksads_sad.csv: 11434
Deleted 203 rows with missing data from mh_p_ksads_sad.csv.
Missing: 729. Number of rows without missing data in mh_y_ksads_gad.csv: 10994
Deleted 729 rows with missing data from mh_y_ksads_gad.csv.
Missing: 679. Number of rows without missing data in mh_y_ksads_sad.csv: 11044
Deleted 679 rows with missing data from mh_y_ksads_sad.csv.
ksads_sad_raw_209_t
0    7872
1    2863
2     309
dtype: int64
Missing: 11. Number of rows without missing data in mh_y_pps.